In [2]:
import requests
import numpy as np
import pandas as pd

start = 1
stop = 111
step = 1
index = pd.Series(range(start, stop, step))

url = "https://statsapi.web.nhl.com/api/v1/teams/"

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop through the URLs
for url_suffix in index.astype(str):
    full_url = url + url_suffix
    
    # Send the request
    r = requests.get(full_url)
    
    # Check if the request was successful
    if r.status_code == 200:
        json_data = r.json()
        
        # Extract the data from the JSON response
        data = json_data.get('teams')
        
        # Normalize the JSON data into a DataFrame
        df = pd.json_normalize(data=data, sep="_")
        
        # Append the DataFrame to the result DataFrame
        result_df = result_df.append(df, ignore_index=True)
    else:
        print(f"Request for URL {full_url} failed with status code {r.status_code}")

col_dict = {"id":"team_id",
            "name":"team_full_name",
            "link":"team_link",
            "abbreviation":"team_abbreviation",
            "teamName":"team_name",
            "locationName":"location_name",
            "firstYearOfPlay":"first_year_of_play",
            "shortName":"team_short_name",
            "officialSiteUrl":"official_site_url",
            "franchiseId":"franchise_id_dup",
            "active":"active",
            "venue_name":"venue_name",
            "venue_link":"venue_link",
            "venue_city":"venue_city",
            "venue_timeZone_id":"venue_time_zone_id",
            "venue_timeZone_offset":"venue_time_zone_offset",
            "venue_timeZone_tz":"venue_time_zone_tz",
            "division_id":"division_id",
            "division_name":"division_name",
            "division_nameShort":"division_name_short",
            "division_link":"division_link",
            "division_abbreviation":"division_abbreviation",
            "conference_id":"conference_id",
            "conference_name":"conference_name",
            "conference_link":"conference_link",
            "franchise_franchiseId":"franchise_id",
            "franchise_teamName":"franchise_team_name",
            "franchise_link":"franchise_link",
            "venue_id":"venue_id"}

# Rename columns
result_df = result_df.rename(columns=col_dict)
result_df.columns = result_df.columns.str.upper()

#FILL ALL NA VALUES WITH EMPTY STRING
result_df = result_df.fillna('')

#CONVERT ALL COLUMNS TO STRING
result_df = result_df.astype(str)

Request for URL https://statsapi.web.nhl.com/api/v1/teams/59 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/68 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/69 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/70 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/71 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/72 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/73 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/74 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/75 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/76 failed with status code 404
Request for URL https://statsapi.web.nhl.com/api/v1/teams/77 failed with status code 404
Request for URL https

In [3]:
import cx_Oracle
import os

# Oracle database connection details
username = os.getenv('Oracle_Username')
password = os.getenv('Oracle_Password')
hostname = '10.0.0.69'
port = '1521'
service_name = 'XEPDB1'

# Construct the connection string
dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Create a cursor object
cursor = connection.cursor()

# Define the target table name
table_name = 'dim_team_load'

# Drop the table if it already exists
drop_query = f"DROP TABLE {table_name}"
try:
    cursor.execute(drop_query)
    print(f"Dropped table {table_name}")
except cx_Oracle.DatabaseError as e:
    error_code, error_message = e.args[0].code, e.args[0].message
    if error_code != 942:  # ORA-00942: table or view does not exist
        print(f"Error occurred while dropping table {table_name}: {error_message}")

   
# Read the SQL file
sql_file = r"C:\Users\Jeremy\Documents\nhl_analysis\02_Source_Code\table_create_scripts\dim_team_load.sql"
with open(sql_file, 'r') as file:
    create_table_query = file.read()

# Create the table
cursor.execute(create_table_query)
print(f"Created table {table_name}")

# Convert dataframe to a list of tuples
data_tuples = [tuple(row) for row in result_df.values]

# Prepare the INSERT query with placeholders for the columns
insert_query = f"INSERT INTO {table_name} " \
               f"({', '.join(result_df.columns)}) " \
               f"VALUES ({', '.join([':' + str(i+1) for i in range(len(result_df.columns))])})"

# Insert data into the target table
cursor.executemany(insert_query, data_tuples)
print(f"Inserted {len(data_tuples)} rows into {table_name}")

# Commit the changes
connection.commit()

# Read the SQL script
update_file = r"C:\Users\Jeremy\Documents\nhl_analysis\02_Source_Code\update_scripts\dim_team_update.sql"
with open(update_file, 'r') as file:
    script = file.read()

# Split the script into individual statements
statements = script.split(';')

# Execute each statement
for statement in statements:
    if statement.strip():  # Skip empty statements
        try:
            cursor.execute(statement)
            rows_affected = cursor.rowcount
            print(f"Rows affected: {rows_affected}")
        except cx_Oracle.DatabaseError as e:
            error, = e.args
            error_message = error.message
            print(f"Error executing statement: {error_message}")

# Commit the changes
connection.commit()

# Drop the table if it already exists
drop_query = f"DROP TABLE {table_name}"
try:
    cursor.execute(drop_query)
    print(f"Dropped table {table_name}")
except cx_Oracle.DatabaseError as e:
    error_code, error_message = e.args[0].code, e.args[0].message
    if error_code != 942:  # ORA-00942: table or view does not exist
        print(f"Error occurred while dropping table {table_name}: {error_message}")

# Close the cursor and the database connection
cursor.close()
connection.close()

Dropped table dim_team_load
Created table dim_team_load
Inserted 87 rows into dim_team_load
Rows affected: 0
Rows affected: 87
Rows affected: 87
Dropped table dim_team_load
